In [ ]:
def model1(input_shape):
    '''
    simple 1D conv model
    '''
    eps = 1e-5

def base_model(input_shape, params_shape, alpha_lock=0.8):
    assert 0 <= alpha_lock <= 1
    img_input = Input(shape=(input_shape), name='img_input')
    params_input = Input(shape=(params_shape), name='params_input')
    xcpn = keras.applications.xception.Xception(include_top=False, weights='imagenet', 
                                         input_shape=None, pooling=None)
    
    net = xcpn(img_input)
    
    for layer in xcpn.layers[:int(len(xcpn.layers)*alpha_lock)]:
        layer.trainable = False
    net = GlobalAveragePooling2D()(net)
    net = concatenate([net, params_input])
    net = Dense(512, activation='linear', kernel_regularizer=l2(eps), bias_regularizer=l2(eps))(net)
    net = LeakyReLU(alpha=0.3)(net)
    net = BatchNormalization()(net)
    net = Dropout(0.001)(net)
    net = Dense(units=50, activation=relu,  kernel_regularizer=l2(eps), bias_regularizer=l2(eps))(net)
    outputs = Dense(units=18,  kernel_regularizer=l2(eps), bias_regularizer=l2(eps))(net)
    
    model = Model(inputs=[img_input, params_input], outputs=outputs)
    
#     for layer in model.layers[:int(len(model.layers)*alpha_lock)]:
#         layer.trainable = False
        
    
    model.compile(
            loss=mean_absolute_error,
            optimizer=Adam(lr = 1e-3),
    )
    
    return model